In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Underweight"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Underweight/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Underweight/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Underweight/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Underweight/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Review subdirectories to find one related to Underweight
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)
print(f"Available TCGA directories: {tcga_dirs}")

# For the "Underweight" trait, we need to find cancer types that commonly report BMI or weight data
# Potential relevant directories might include:
# - Stomach/GI cancers (weight loss is common)
# - Pancreatic cancer (often associated with weight loss)
# - Lung cancer (may have cachexia/weight loss data)

# Initialize variables to track potential matches
relevant_dirs = []
for dir_name in tcga_dirs:
    dir_lower = dir_name.lower()
    if any(term in dir_lower for term in ["stomach", "pancreatic", "lung", "liver"]):
        relevant_dirs.append(dir_name)

print(f"Potential relevant directories for {trait}: {relevant_dirs}")

# If we found potentially relevant directories, choose one with priority to stomach/pancreatic
selected_dir = None
if relevant_dirs:
    # Prioritize pancreatic cancer as it's often associated with significant weight loss
    for dir_name in relevant_dirs:
        if "pancreatic" in dir_name.lower():
            selected_dir = dir_name
            break
    # If no pancreatic cancer, try stomach cancer
    if not selected_dir:
        for dir_name in relevant_dirs:
            if "stomach" in dir_name.lower():
                selected_dir = dir_name
                break
    # If still no match, take the first relevant directory
    if not selected_dir and relevant_dirs:
        selected_dir = relevant_dirs[0]

if selected_dir:
    print(f"Selected directory for {trait}: {selected_dir}")
    
    # Get the full path to the directory
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Find clinical and genetic data files
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {clinical_file_path}")
    print(f"Genetic data file: {genetic_file_path}")
    
    # Step 3: Load the data files
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Step 4: Print column names of clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Check for weight-related columns
    weight_related_cols = [col for col in clinical_df.columns if any(term in col.lower() for term in ["weight", "bmi", "body", "mass"])]
    print("\nWeight-related columns found:")
    print(weight_related_cols)
    
    # Check if both datasets are available and if weight data exists
    is_gene_available = not genetic_df.empty
    is_trait_available = not clinical_df.empty and len(weight_related_cols) > 0
    
    # Initial validation
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
else:
    print(f"No directory specifically relevant to the trait: {trait}")
    
    # Since the trait is not directly represented, we should record this fact
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print(f"Task marked as completed. {trait} is not directly represented in the TCGA dataset.")


Available TCGA directories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_


Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'adenocarcinoma_invasion', 'age_at_initial_pathologic_diagnosis', 'alcohol_history_documented', 'alcoholic_exposure_category', 'amount_of_alcohol_consumption_per_day', 'anatomic_neoplasm_subdivision', 'anatomic_neoplasm_subdivision_other', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_diabetes_onset', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_pancreatitis_onset', 'family_history_of_cancer', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'frequency_of_alcohol_consumption', 'gender', 'histologic_grading_tier_category', 'histolo

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate age and gender columns
candidate_age_cols = ["age_at_initial_pathologic_diagnosis", "days_to_birth"]
candidate_gender_cols = ["gender"]

# Load the clinical data file path
clinical_file_path, _ = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, "TCGA_Pancreatic_Cancer_(PAAD)"))

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')

# Preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [65, 48, 75, 71, 70], 'days_to_birth': [-23962, -17794, -27600, -26028, -25920]}

Gender columns preview:
{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Inspect age columns
age_cols = {'age_at_initial_pathologic_diagnosis': [65, 48, 75, 71, 70], 
            'days_to_birth': [-23962, -17794, -27600, -26028, -25920]}

# Inspect gender columns
gender_cols = {'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

# Select the appropriate columns
# For age, choose age_at_initial_pathologic_diagnosis as it directly gives the age in years
# days_to_birth would need conversion (dividing by 365.25 and taking absolute value)
age_col = 'age_at_initial_pathologic_diagnosis' if age_cols else None

# For gender, we have only one option which appears to have valid values
gender_col = 'gender' if gender_cols and len(gender_cols) > 0 else None

# Print the chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Pancreatic Cancer directory identified in Step 1
selected_dir = "TCGA_Pancreatic_Cancer_(PAAD)"
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check for the presence of weight-related columns
weight_related_cols = [col for col in clinical_df.columns if any(term in col.lower() for term in ["weight", "bmi", "body", "mass"])]
print(f"Weight-related columns found: {weight_related_cols}")

# Create a function to convert weight data to an underweight binary classification
def create_underweight_feature(df):
    """Create a binary underweight feature from initial_weight column"""
    # Check if initial_weight column exists
    if 'initial_weight' not in df.columns:
        print("No initial_weight column found")
        # Use sample IDs as fallback (less accurate, but following the dataset structure)
        return df.index.to_series().apply(tcga_convert_trait)
    
    # Extract initial weight values and convert to numeric
    weights = pd.to_numeric(df['initial_weight'], errors='coerce')
    
    # Define underweight threshold (generally BMI < 18.5, but without height data, 
    # we'll use a reasonable threshold of 50kg for adults)
    underweight_threshold = 50  # kg
    
    # Create binary classification (1 = underweight, 0 = not underweight)
    underweight = (weights < underweight_threshold).astype(int)
    
    print(f"Created underweight feature with {underweight.sum()} underweight patients out of {len(underweight)} total")
    return underweight

# Create the underweight trait
underweight_feature = create_underweight_feature(clinical_df)

# Create a DataFrame with the clinical features
feature_list = []
# Explicitly name the trait column correctly
feature_list.append(underweight_feature.rename(trait))

if age_col:
    age_data = clinical_df[age_col].apply(tcga_convert_age).rename("Age")
    feature_list.append(age_data)
if gender_col:
    gender_data = clinical_df[gender_col].apply(tcga_convert_gender).rename("Gender")
    feature_list.append(gender_data)

clinical_features = pd.concat(feature_list, axis=1)

# Verify the trait column exists with the correct name
print(f"Clinical features columns: {clinical_features.columns.tolist()}")
if trait not in clinical_features.columns:
    print(f"Warning: '{trait}' column not found, adding it explicitly")
    clinical_features[trait] = underweight_feature.values

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Verify trait column exists in linked_data
print(f"First few columns in linked_data: {linked_data.columns[:5].tolist()}")
if trait not in linked_data.columns:
    print(f"ERROR: '{trait}' column not found in linked_data")
    # Try to fix by adding it if we still have the original data
    if clinical_features is not None and trait in clinical_features.columns:
        common_indices = set(linked_data.index).intersection(set(clinical_features.index))
        linked_data[trait] = clinical_features.loc[common_indices, trait]
        print(f"Added '{trait}' column from clinical_features")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for pancreatic cancer patients from TCGA. The 'Underweight' trait was derived from the initial_weight column."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Weight-related columns found: ['followup_case_report_form_submission_reason', 'initial_weight']
Created underweight feature with 9 underweight patients out of 196 total
Clinical features columns: ['Underweight', 'Age', 'Gender']
Saved clinical data with 196 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (183, 19851) (samples x features)
First few columns in linked_data: ['Underweight', 'Age', 'Gender', 'A1BG', 'A1BG-AS1']


After handling missing values, data shape: (183, 19851)
For the feature 'Underweight', the least common label is '1' with 9 occurrences. This represents 4.92% of the dataset.
The distribution of the feature 'Underweight' in this dataset is fine.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 65.0
  75%: 73.0
Min: 35
Max: 88
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 82 occurrences. This represents 44.81% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved usable linked data to ../../output/preprocess/Underweight/TCGA.csv
